# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [5]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [6]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [7]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010 driving for Red Bull Racing. It was his first world championship title in Formula 1.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [8]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [9]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 Formula 1 championship was won by Fernando Alonso, driving for the Renault F1 Team.


In [10]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton from the Mercedes team.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [11]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [16]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [20]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Fernando Alonso won the F1 championship in 2006.


Few Shots for classification.


In [21]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# classifying

In [31]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in knowing what is the mood of the human and classifying them as happy or sad.

     I finished my homeworks early and I have sometimes to play with my friends.
     Sentiment: happy

     I could not finish my all workouts today because I did not finish my breakfast.
     Sentiment: sad.

     I have to finish my workouts early so I could do my howeworks.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("i could not sumbit all my homeworks today.", context_user))
# try these: 
# "I left the work early so now i have time to meet my brother."
# "I have a alot of homewroks today so i could not go to the gym."
# (you must try it LOL :)) "sometimes i wish if the day has 28 hours so i could manage to do all of my list of things must to do"


Sentiment: sad


# you are an Expert of..

In [51]:
context_user = [
    {'role':'system', 'content':"""You are an expert in Saudi league.
    You are going to answer the question of the user giving the name of the team,
    the last year the team has won the leage and the number of the leagues the team have, following the format:
    Team:
    Champion in year:
    Total leagues: """
    }
]

print(return_OAIResponse("Al-Hilal", context_user))

Team: Al-Hilal
Champion in year: 2020
Total leagues: 17


In [50]:
context_user = [
    {'role':'system', 'content':"""You are an expert in Oscars.
    You are going to answer the question of the user giving the number of the year,
    the best movie and the best actor and the best Director  and the best Actress, following the format:
    best movie:
    best actor:
    best Director
    best Actress:  """
    }
]

print(return_OAIResponse("2021", context_user))

best movie: Nomadland
best actor: Anthony Hopkins
best Director: Chloé Zhao
best Actress: Frances McDormand


In [53]:
context_user = [
    {'role':'system', 'content':"""You are an expert in Cars.
    You are going to answer the question of the user giving the name of the  car brand,
    the best car for OFFROAd and the best car inside city and the best car for Travelling and the best Economy car, following the format:
    OFFROAd:
    inside city:
    Travelling:
    Economy:  """
    }
]

print(return_OAIResponse("Toyta", context_user))

I believe you are referring to the car brand Toyota.

OFFROAd: Toyota Land Cruiser 
Inside city: Toyota Corolla 
Travelling: Toyota Camry 
Economy: Toyota Prius


# Insights:


### Summary:
#### The AI performed well in sentiment classification and knowledge-based queries. However, some inconsistencies were observed.
Issues Found:

    -Hallucinations: generated incorrect details in responses.
    -Formatting Errors: Some outputs did not strictly follow the expected format.
    -Misclassification: Sentiment analysis sometimes failed on ambiguous statements

# I learned
### Providing clear examples improves response accuracy and few-shot learning helps structure outputs consistently.